# Introduction
The following code for NSE has been picked up from [Ewald's examples](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb) given below

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import product

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=12)

<IB connected to 127.0.0.1:4004 clientId=12>

In [153]:
# Get the list of Equity Option scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Replace & for M&M, L&T, etc. This is not needed for IBKR
mask = paisa.loc[paisa.Symbol.str.contains("&"), "Symbol"]
paisa.loc[paisa.Symbol.isin(mask), "Symbol"] = mask.str.replace("&", "")

# Add columns
paisa["iSymbol"] =  paisa.Symbol.tolist()  # For IBKR scrip names
paisa["Exchange"] = 'NSE'   # for the Exchange

# Exclusions and Inclusions
vix_nifty = ['VIX', 'NIFTY']     # mask for Index
symbol_rename = {'NIFTYCPSE': 'CPSE', 'NIFTY': 'NIFTY50'}   # renamed for IBKR
symbol_del = ['INDIAVIX']

# Build the Index dataframe
index_mask = paisa.Symbol.str.contains('|'.join(vix_nifty))
nse_ind = paisa.loc[index_mask, :] # Keep only VIX and NIFTY

nse_ind = nse_ind.loc[~nse_ind.Symbol.str.contains('|'.join(symbol_del)), :] # INDIAVIX not in IBKR
nse_ind = nse_ind.replace({"iSymbol": symbol_rename}) # Rename scrips to align with IBKR

# Qualify the Index
q_idx = list(map(lambda s, e : Index(symbol=s, exchange=e), 
                             nse_ind.iSymbol, nse_ind.Exchange))
qualIndex = ib.qualifyContracts(*q_idx)

# Build the Stock dataframe
nse_stk = paisa.loc[~index_mask, :].reset_index(drop=True) # Not the index mask!

# Shorten long scrip names in line with IBKR
iSymbol = nse_stk.Symbol.str.slice(0,9)
nse_stk.loc[:, "iSymbol"] = iSymbol

# Qualify the Stock
q_stk = list(map(lambda s, e: Stock(symbol=s, exchange=e),
                                      nse_stk.iSymbol, nse_stk.Exchange))
qualStock = ib.qualifyContracts(*q_stk)

In [165]:
# Make a single list of qualified contracts
qualified = [(i.symbol, i) for i in qualStock]+[(i.symbol, i) for i in qualIndex]

df_qualed = pd.DataFrame(qualified, columns=["iSymbol", "Contract"])

In [166]:
df_paisa = pd.concat([nse_stk, nse_ind], ignore_index=True, sort=True)

In [167]:
pd.merge(df_qualed, df_paisa, on="iSymbol")

,iSymbol,Contract,Exchange,ExpMgn%,ExpMgnPerLt,ExpMgnPerShr,Mlot,SpMgn%,SpMgnPerLt,SpMgnPerShare,Symbol,TotMgn%,TotMgnPerLt,TotMgnPerShr
0,PCJEWELLE,"Stock(conId=119653211, symbol='PCJEWELLE', exc...",NSE,29.83,37652.9175,25.1019,1500,31.24,39435.0,26.29,PCJEWELLER,61.07,77087.9175,51.3919
1,JISLJALEQ,"Stock(conId=80423707, symbol='JISLJALEQ', exch...",NSE,30.00,217620.0000,24.1800,9000,12.50,90720.0,10.08,JISLJALEQS,42.50,308340.0000,34.2600
2,JPASSOCIA,"Stock(conId=56984254, symbol='JPASSOCIA', exch...",NSE,22.98,118369.9800,3.4815,34000,14.65,75480.0,2.22,JPASSOCIAT,37.63,193849.9800,5.7015
3,ADANIENT,"Stock(conId=56986798, symbol='ADANIENT', excha...",NSE,20.00,158840.0000,39.7100,4000,16.75,133040.0,33.26,ADANIENT,36.75,291880.0000,72.9700
4,RAYMOND,"Stock(conId=56984932, symbol='RAYMOND', exchan...",NSE,27.50,185724.0000,232.1550,800,8.05,54424.0,68.03,RAYMOND,35.55,240148.0000,300.1850
5,INDIACEM,"Stock(conId=56985582, symbol='INDIACEM', excha...",NSE,22.50,89656.8750,25.6163,3500,12.92,51520.0,14.72,INDIACEM,35.42,141176.8750,40.3363
6,DHFL,"Stock(conId=56984051, symbol='DHFL', exchange=...",NSE,27.50,258637.5000,172.4250,1500,7.51,70710.0,47.14,DHFL,35.01,329347.5000,219.5650
7,GRANULES,"Stock(conId=186878917, symbol='GRANULES', exch...",NSE,20.00,98700.0000,19.7400,5000,12.05,59500.0,11.90,GRANULES,32.05,158200.0000,31.6400
8,RELCAPITA,"Stock(conId=56984773, symbol='RELCAPITA', exch...",NSE,20.00,118185.0000,78.7900,1500,11.13,65820.0,43.88,RELCAPITAL,31.13,184005.0000,122.6700
9,TV18BRDCS,"Stock(conId=56985560, symbol='TV18BRDCS', exch...",NSE,20.00,83300.0000,9.8000,8500,10.91,45475.0,5.35,TV18BRDCST,30.91,128775.0000,15.1500


In [104]:
def get_chains(scrips):
    '''Gets the option chains for the scrips passed
       Args:
          (scrips): list: qualified list of equity or index options with conId
       Returns:
          list of option chain'''
    
    # Prepare a sequence for ib.reqSecDefOptParams
    seq = [(i.symbol, '', i.secType, i.conId) for i in scrips]

    # build the chains
    chains = [ib.reqSecDefOptParams(*p) for p in seq]
    # list(map(lambda p: ib.reqSecDefOptParams(*p), seq))  # Another way of doing!
    
    return chains

In [105]:
# Get the stock chains
stock_chains = get_chains(qualStock[0:3]) # !data_limiter

In [106]:
# Get the stocks list
stocks = [(i.tradingClass, i.expirations, i.strikes) for elem in stock_chains for i in elem]

In [107]:
# Build the dataframe (Ref: Stackoverflow: 51579138)
L = [[[x[0]], sorted(x[1]), sorted(x[2])] for x in stocks]
df1 = pd.DataFrame([j for i in L for j in product(*i)], columns=['Scrip','Expiry','Strike'])

In [ ]:
# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

# Get the DTE
df1["DTE"] = [get_dte(x) for x in df1.Expiry]

In [ ]:
# function to get standard deviation
def get_stdev(contract, dte):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
    Returns:
       standard deviation in days (int)
    '''
    sd_days = str(dte)+' D'
    bars = ib.reqHistoricalData(contract=scrip, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    return sd

# Get the Standard Deviation

In [ ]:
[]

In [ ]:
a = [{1, 2, 3}, {4, 5}]

In [ ]:
type(a[0])

In [ ]:
strike

In [ ]:
[get_dte(i) for elem in expiry for i in elem]

In [ ]:
import itertools
scrip = ['a', 'b']
strike = [1, 2, 3, 4]
expiry = ['x', 'y', 'z']

nest = [scrip, strike, expiry]
list(itertools.product(*nest))

In [ ]:

pd.DataFrame(())
list(zip(scrip, expiry, strike))

In [ ]:
# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

In [ ]:
[get_dte(i) for elem in expiry for i in elem]

In [ ]:
a = []
for element in expiry:
    for dt in element:
        a.append(get_dte(dt))
a

In [ ]:
my_list = []
for element in chains:
    for chain in element:
        my_list.append(chain.strikes)
        
my_list

In [ ]:
[(chain.tradingClass, chain.strikes) for chain in element for element in chains]

In [ ]:
for elems in chains:
    for strikes in elems:
        strikez = list(strikes.strikes)


In [ ]:
chains

In [ ]:
{'Class':[i.tradingClass for c in chains for i in c], 'Strikes':[i.strikes for c in chains for i in c]}

In [ ]:
len(chains[0][0][5])

In [ ]:
chains

In [ ]:
for xyz in zip(*chains):
    print(*xyz)

In [ ]:
# instantiate stock contract class
scrip = Stock(symbol='WOCKPHARM', exchange='NSE' )
ib.qualifyContracts(scrip)

In [ ]:
# get the underlying's option chain with security definition parameter
chains = ib.reqSecDefOptParams(underlyingSymbol=scrip.symbol, futFopExchange = '', 
                               underlyingSecType=scrip.secType, underlyingConId=scrip.conId)

# put that in a dataframe
chains_df = util.df(chains)
chains_df.head()

In [ ]:
# make arrays for strikes and expiries
strikes = np.array(list(*chains_df.strikes))
expiries = np.array(list(*chains_df.expirations))

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

dte = list(map(get_dte, expiries))

# function to get standard deviation
def get_stdev(contract, dte):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
    Returns:
       standard deviation in days (int)
    '''
    sd_days = str(dte)+' D'
    bars = ib.reqHistoricalData(contract=scrip, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    return sd

stdev = list(map(lambda x, y: get_stdev(x, y), np.repeat(scrip, len(dte)), dte))

# equalize lengths of array for strikes+expiries+dte
a_strikes = np.repeat(strikes, len(expiries))
a_expiries = np.tile(expiries, len(strikes))   # tile to repeat expiries in blocks
a_dte = np.tile(dte, len(strikes))
a_stdev = np.tile(stdev, len(strikes))

# create a dataframe of Strike and Expiry
strikes_df = pd.DataFrame({'Strike': a_strikes, 'Expiry': a_expiries, 'DTE': a_dte, 'Stdev': a_stdev})

# insert a key and merge strikes and chains
strikes_df['key'] = 0
chains_df['key'] = 0

raw_chain = pd.merge(strikes_df, chains_df, on='key')
raw_chain = raw_chain.drop(['expirations', 'strikes', 'key'], axis=1)
raw_chain.head()

In [ ]:
contract = Option(symbol=scrip.symbol, exchange=scrip.exchange, 
                  lastTradeDateOrContractMonth='20180726', strike='640', right='P')
order = Order(action='SELL', orderType='MKT', totalQuantity=100.0)

In [ ]:
try:
    ib.whatIfOrder(contract, order)
except Exception as inst:
    print(inst.args)

In [ ]:
ib.whatIfOrder(contract, order)

In [ ]:
# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY from paisa
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

paisa = paisa.loc[paisa.Symbol.isin(symbol), :]

# Make paisa's symbol compatible with IBKR symbol
paisa.loc[:, "Symbol"] = symbol.str.slice(0,9)

# Replace & for M&M, L&T, etc
mask = paisa.loc[paisa.Symbol.str.contains('&'), "Symbol"]
paisa.loc[paisa.Symbol.isin(mask), "Symbol"] = mask.str.replace('&','')

In [ ]:
paisa

In [ ]:
help(ib.events)

In [ ]:
[ticker] = ib.reqTickers(contract)

In [ ]:
ib.reqContractDetails(contract)

In [ ]:
ib.tickers()

# Original code of Ewald

In [ ]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

In [ ]:
# Get the ticker
[ticker] = ib.reqTickers(spx)
ticker

In [ ]:
spxValue = ticker.marketPrice()

In [ ]:
spxValue

In [ ]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
util.df(chains)

In [ ]:
scrip = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(scrip)

In [ ]:
scrip_chain = ib.reqSecDefOptParams(scrip.symbol, '', scrip.secType, scrip.conId)
util.df(scrip_chain)

In [ ]:
scrip_chain

In [ ]:
help(Contract.right)